In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
# check the data
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# check some text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# check unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


# Preprocess data

### vectorize text

In [6]:
# mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}

# mapping from indices to chars
idx2char = np.array(vocab)

# convert text data to integers
text_as_int = np.array([char2idx[c] for c in text])

In [7]:
# check the character to index mapping
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [8]:
# check mapping of the text
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


### create training data

In [9]:
# max sequence length
seq_length = 100

examples_per_epoch = len(text)//(seq_length+1)

# create dataset
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# check dataset
for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [10]:
# use batching to convert individual indices to a sequence
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# check sequence
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [11]:
# for each sequence, shift one to create the label
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
# check the dataset
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [13]:
# check logic
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [14]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

# Create model

In [15]:
# each vocab is a character
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [17]:
model = build_model(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [19]:
# check output
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [20]:
# test model on a sample
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=1).numpy()

print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "your shame, by this unholy braggart,\n'Fore your own eyes and ears?\n\nAll Conspirators:\nLet him die fo"

Next Char Predictions: 
 "q?q$g RHbPPN$iODcLsTKHhfThBi 'fQJ!VbsPZnSGeQMQKpemcfQb-La!.N?TGzgYNk- ecvT. -nrIXm!X nMBY!z,qbL\nHLEr"


# Train

In [21]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1745925


In [22]:
model.compile(optimizer='adam', loss=loss)

In [23]:
# save checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [24]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 18s 104ms/step - loss: 2.6697
Epoch 2/10
172/172 [==============================] - 18s 104ms/step - loss: 1.9589
Epoch 3/10
172/172 [==============================] - 18s 104ms/step - loss: 1.6925
Epoch 4/10
172/172 [==============================] - 18s 104ms/step - loss: 1.5427
Epoch 5/10
172/172 [==============================] - 18s 104ms/step - loss: 1.4545
Epoch 6/10
172/172 [==============================] - 18s 104ms/step - loss: 1.3943
Epoch 7/10
172/172 [==============================] - 18s 104ms/step - loss: 1.3498
Epoch 8/10
172/172 [==============================] - 18s 105ms/step - loss: 1.3106
Epoch 9/10
172/172 [==============================] - 18s 106ms/step - loss: 1.2751
Epoch 10/10
172/172 [==============================] - 18s 106ms/step - loss: 1.2423


# Generate text

In [25]:
# the model only takes fixed batch size once built
# if want to run model with different batch size, need to rebuild model and store weights
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [26]:
def generate_text(model, start_string):
    # number of characters to generate
    num_generate = 1000

    # convert input string to integers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    # low temperature for more predictable text, high vice versa
    temperature = 1.0

    # batch size = 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # use categorical distribution to predict character
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # pass predicted character and previous hidden state as next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [27]:
print(generate_text(model, start_string=u'ROMEO: '))

ROMEO: 'tis there.
Go tell you, prompts it.

KING HENRY VI:
I will, indeed to say; I fear
We make road heart; and of his mouth'd dram not.

TYNREL:
He doth short them would here you in the orderves.
The harl: thou camer with in that Inemarity
His friends are constant to cry to Rome, that justice, or how, or foe
Than appear the rest? I will not hear me now,
And I fetch the boldly Belunkin'd. Now I can leave.

EDWARD:
Nothand seems.
What worthy truck out on a time-oast,
Of that hour for Rome are tor mer licks.
Fum to their in his admits of a tedious.
A king of kinghood for you to appear,' quarce are humour
within your heart
Prick'd once.

SICINIUS:
Good mock-grace.

Lord:
His vaultiel the misery remors.

HORTENSIO:
Yes, sir; well, we'll not hell.

NORFOLK:
Her certain!

SICINIUS:
Now, fall, so too; shall see thee, I say!
Take thou to jest; treading him, or seeming kind his graces
Of that heartily were lodgiest
Master of all this world's pallan shall Look on her.

PETRUCHIO:
They'll we wo

# Custom training

In [28]:
model = build_model(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

In [29]:
optimizer = tf.keras.optimizers.Adam()

@tf.function
def train_step(input, target):
    with tf.GradientTape() as tape:
        predictions = model(input)
        loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(target, predictions, from_logits=True))

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [30]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    # reset hidden state at every epoch
    model.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)

        if batch_n % 100 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch + 1, batch_n, loss))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, loss))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.174837589263916
Epoch 1 Batch 100 Loss 2.371006965637207
Epoch 1 Loss 2.1392
Time taken for 1 epoch 18.51416540145874 sec

Epoch 2 Batch 0 Loss 2.144247055053711
Epoch 2 Batch 100 Loss 1.8878687620162964
Epoch 2 Loss 1.7631
Time taken for 1 epoch 17.27969002723694 sec

Epoch 3 Batch 0 Loss 1.7810107469558716
Epoch 3 Batch 100 Loss 1.6371573209762573
Epoch 3 Loss 1.5847
Time taken for 1 epoch 17.39772319793701 sec

Epoch 4 Batch 0 Loss 1.5576701164245605
Epoch 4 Batch 100 Loss 1.5388844013214111
Epoch 4 Loss 1.5136
Time taken for 1 epoch 17.336246252059937 sec

Epoch 5 Batch 0 Loss 1.4515957832336426
Epoch 5 Batch 100 Loss 1.4678282737731934
Epoch 5 Loss 1.4833
Time taken for 1 epoch 17.685662508010864 sec

Epoch 6 Batch 0 Loss 1.3714306354522705
Epoch 6 Batch 100 Loss 1.455464482307434
Epoch 6 Loss 1.3793
Time taken for 1 epoch 17.329514026641846 sec

Epoch 7 Batch 0 Loss 1.313724160194397
Epoch 7 Batch 100 Loss 1.32773756980896
Epoch 7 Loss 1.3541
Time taken for